In [11]:
import igraph as ig  
import pandas as pd
import json
import random

In [12]:
nodes_path = "../file_data/fb-pages-food.nodes"
edges_path = "../file_data/fb-pages-food.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [13]:
g = ig.Graph()
g.add_vertices(nodes_df["new_id"].tolist())
g.add_edges(edges_df[["source", "target"]].values.tolist())
g.vs["name"] = nodes_df["name"].tolist()

In [14]:
communities = g.community_edge_betweenness()
communities = communities.as_clustering()

In [15]:
def generate_community_names(num_cliques):
    prefixes = [
        "Neo",
        "Alt",
        "Hyper",
        "Meta",
        "Quantum",
        "Terra",
        "Cyber",
        "Omega",
        "Solar",
        "Astra",
    ]
    roots = [
        "Gen",
        "Net",
        "Sphere",
        "Cluster",
        "Link",
        "Wave",
        "Flow",
        "Pulse",
        "Code",
        "Echo",
    ]
    suffixes = ["ium", "on", "os", "ix", "ia", "or", "an", "is", "us", "er"]

    community_names = set()

    while len(community_names) < num_cliques:
        name = (
            f"{random.choice(prefixes)}{random.choice(roots)}{random.choice(suffixes)}"
        )
        name += f" {random.randint(1000, 9999)}"
        if name not in community_names:
            community_names.add(name)
    print("nomi generati")
    return list(community_names)

In [16]:
def assign_community_names_to_graph(graph, communities):
    num_communities = len(communities)
    community_names = generate_community_names(num_communities)
    updated_communities = []

    if "aggregation" not in graph.vs.attributes():
        graph.vs["aggregation"] = [[] for _ in range(graph.vcount())]

    if "community_name" not in graph.vs.attributes():
        graph.vs["community_name"] = [None for _ in range(graph.vcount())]

    for i, community in enumerate(communities):
        community_name = community_names[i]
        updated_communities.append({"name": f"{community_name}", "nodes": community})
        for node in community:
            graph.vs[node]["aggregation"].append(f"{community_name}")
            if graph.vs[node]["community_name"] is None:
                graph.vs[node]["community_name"] = community_name
    if "community_name" not in graph.es.attributes():
        graph.es["community_name"] = [None] * graph.ecount()
    for edge in graph.es:
        source_node = edge.source  
        graph.es[edge.index]["community_name"] = graph.vs[source_node]["community_name"]

    return updated_communities

In [17]:
updated_communities = assign_community_names_to_graph(g, communities)


nomi generati


In [48]:
def create_community_graph(graph, communities):
    communityGraph = ig.Graph()
    link_map = {}  
    for community in communities:
        community_name = community["name"]
        children = [v.index for v in graph.vs if v["community_name"] == community_name]
        communityGraph.add_vertex(
            name=community_name,
            id=community_name,
            children=children,
        )
    for edge in graph.es:
        source_community = graph.vs[edge.source]["community_name"]
        target_community = graph.vs[edge.target]["community_name"]
        if source_community != target_community:
            link_key = f"{source_community}-{target_community}"
            reverse_link_key = f"{target_community}-{source_community}"
            if link_key not in link_map and reverse_link_key not in link_map:
                communityGraph.add_edge(
                    source_community, target_community, id=link_key
                )
                link_map[link_key] = True

    return communityGraph

In [51]:
community_graph = create_community_graph(g, updated_communities)
for edge in community_graph.es:
    print(edge['id'])

MetaWaveer 2757-CyberLinkos 2267
SolarNeton 8333-CyberLinkos 2267
TerraLinkis 4537-AstraLinkos 9217
NeoCodeor 5004-CyberLinkos 2267
NeoNeton 1643-QuantumNetus 9579
CyberLinkos 2267-HyperClusterer 3334
CyberLinkos 2267-SolarPulseis 9371
CyberLinkos 2267-SolarLinker 5750
AstraLinkos 9217-SolarPulseis 9371
SolarPulseis 9371-NeoEchoia 7504
AltLinkis 7967-CyberLinkos 2267
HyperCodeia 7518-AltLinkis 7967
SolarLinker 5750-AstraCodeis 5274
MetaClusteros 8551-NeoNeton 1643
CyberLinkos 2267-NeoNeton 1643
HyperCodeia 7518-CyberLinkos 2267
CyberLinkos 2267-AstraLinkos 9217
SolarLinker 5750-AstraLinkos 9217
CyberLinkos 2267-MetaClusteros 8551
QuantumPulsean 2090-MetaClusteros 8551
AstraPulseor 8055-CyberLinkos 2267
NeoNeton 1643-SolarSphereer 9735
CyberLinkos 2267-AltCodeos 4790
AstraEchous 3755-CyberLinkos 2267
CyberLinkos 2267-OmegaWaveor 9574
CyberPulseos 1403-CyberLinkos 2267
SolarLinker 5750-TerraSphereus 9666
CyberSpherean 6391-CyberLinkos 2267
CyberLinkos 2267-TerraLinkis 4537
AstraEchous 37

In [52]:
output_data = {
    "graph": {
        "nodes": [
            {
                "id": node.index,
                "name": node["name"],
                "community_name": node["community_name"],
                "aggregation": node["aggregation"],
            }
            for node in g.vs
        ],
        "edges": [{"source": edge.source, "target": edge.target, "community_name": edge["community_name"]} for edge in g.es],
    },
    "communities": [community for community in updated_communities],
    "community_graph": {
        "nodes": [
            {
                "id": node["id"],
                "name": node["name"],
                "children": node["children"],
            }
            for node in community_graph.vs
        ],
        "edges": [{"source": edge.source, "target": edge.target} for edge in community_graph.es],
    }
}

with open("../graph_data/graph_data_communities.json", "w") as f:
    json.dump(output_data, f, indent=4)